In [5]:
import pandas as pd
import numpy as np
# Load dataset from .parquet file
df = pd.read_csv("dataset/final_data.csv")

# Display first few rows
print(df.head())


                    description  score emoji
0                 grinning face   1.00     😀
1  smiling face with open mouth   1.00     😃
2                  winking face   1.00     😉
3                    robot face   0.50     🤖
4              father christmas   0.75    🎅🏻


In [ ]:
from transformers import BertTokenizer, BertModel
import torch

# Load BERT model and tokenizer
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
bert_model = BertModel.from_pretrained("bert-base-uncased")

# Function to get text embeddings
def get_bert_embedding(text):
    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True, max_length=50)
    outputs = bert_model(**inputs)
    return outputs.last_hidden_state[:, 0, :].detach().numpy().squeeze()  # Take only CLS token


# Apply to dataset
df["text_embedding"] = df["description"].apply(get_bert_embedding)


/var/home/ujjain/.local/lib/python3.12/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [ ]:
from gensim.models import KeyedVectors

# Load Emoji2Vec pre-trained embeddings
emoji2vec_model = KeyedVectors.load_word2vec_format("emoji2vec/pre-trained/emoji2vec.bin", binary=True)

# Function to get emoji embeddings
def get_emoji_embedding(emoji):
    try:
        return emoji2vec_model[emoji]
    except KeyError:
        return np.zeros((300,))  # 300D Zero vector if emoji not in vocab


# Apply to dataset
df["emoji_embedding"] = df["emoji"].apply(get_emoji_embedding)


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

# Convert embeddings to NumPy arrays
X_text = np.stack(df["text_embedding"].values)
X_emoji = np.stack(df["emoji_embedding"].values)

# Concatenate text & emoji embeddings
X = np.hstack((X_text, X_emoji))
y = df["score"].values

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train a regression model
model = RandomForestRegressor(n_estimators=100)
model.fit(X_train, y_train)

# Evaluate performance
y_pred = model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
print(f"Mean Squared Error: {mse:.4f}")


In [ ]:
import pickle

# Save the model
with open("emoji_sentiment_RFR.pkl", "wb") as f:
    pickle.dump(model, f)

print("Model saved successfully!")


# Hyperparameter Tuning for Sentiment Model 

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestRegressor

# Define hyperparameter grid
param_grid = {
    'n_estimators': [100, 200, 300],  
    'max_depth': [None, 10, 20],  
    'min_samples_split': [2, 5, 10],  
    'min_samples_leaf': [1, 2, 4]
}


In [ ]:
# Initialize the model
rf = RandomForestRegressor(random_state=42)

# Perform Grid Search
grid_search = GridSearchCV(rf, param_grid, cv=5, scoring='neg_mean_squared_error', n_jobs=-1)
grid_search.fit(X_train, y_train)

# Best parameters
print("Best Hyperparameters:", grid_search.best_params_)

# Get the best model
best_rf = grid_search.best_estimator_

# Evaluate on Test Set
y_pred = best_rf.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
print("New MSE after tuning:", mse)
